In [1]:
# !git clone https://github.com/Sharp-4rth/temporal_belief_analysis.git

In [ ]:
# For colab:
from google.colab import drive
drive.mount('/content/drive')

# imports:
from temporal_belief_analysis.src.temporal_belief.models.bart_classifier import BARTZeroShotClassifier
from temporal_belief_analysis.src.temporal_belief.utils.config import TOPIC_STANCE_HYPOTHESES, GENERAL_STANCE_HYPOTHESES, ProjectConfig

# Restart after installing:
!pip install convokit

In [2]:
# import unsloth
# import unsloth_zoo
from convokit import Corpus, download
import convokit
# corpus = Corpus(filename=download("subreddit-PoliticalDiscussion"))
corpus = Corpus(filename="/Users/leonidas/.convokit/saved-corpora/PoliticalDiscussion_test_100conv_20250618_110426")
# corpus = Corpus(filename="Users/leonidas/GitHub/temporal_belief_analysis/data/reddit-corpus-small")

In [4]:
# For runpod-jupyter or local
import sys
import os

# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from temporal_belief.models.bart_classifier import BARTZeroShotClassifier
from temporal_belief.utils.config import TOPIC_STANCE_HYPOTHESES, GENERAL_STANCE_HYPOTHESES, ProjectConfig

In [5]:
def prepare_text(text):
    clean_text = text.strip()
    if len(clean_text) > 500:
        clean_text = clean_text[:500] + "..."
    return clean_text

In [6]:
def get_contextual_framing_for_topic(topic, text):
    if topic:
        contextual_text = f"In a discussion about {topic}, this comment states: {text}"
    else:
        contextual_text = f"In a political discussion, this comment states: {text}"
    return contextual_text

In [7]:
# Helper function. Return candidate stances based on the topic:
# Pass in the topic
def get_stance_hypotheses_for_topic(topic):
     # I have the topic already so I don't need these bellow. I need to pass my topic and return the intensity:candidate stances
    topic_lower = topic.lower() if topic else ""
    if any(keyword in topic_lower for keyword in ['abortion', 'reproductive', 'pro-choice', 'pro-life']):
        return TOPIC_STANCE_HYPOTHESES['abortion and reproductive rights']
    elif any(keyword in topic_lower for keyword in ['gun', 'firearm', 'second amendment', 'weapon']):
        return TOPIC_STANCE_HYPOTHESES['gun rights and control']
    elif any(keyword in topic_lower for keyword in ['immigration', 'border', 'refugee', 'immigrant']):
        return TOPIC_STANCE_HYPOTHESES['immigration']
    elif any(keyword in topic_lower for keyword in ['healthcare', 'medical', 'insurance', 'medicare']):
        return TOPIC_STANCE_HYPOTHESES['healthcare']
    elif any(keyword in topic_lower for keyword in ['climate', 'environment', 'carbon', 'global warming']):
        return TOPIC_STANCE_HYPOTHESES['climate change']
    else:
        # Use general hypotheses for unknown topics
        return GENERAL_STANCE_HYPOTHESES

In [4]:
import re
# Helper function:

def mark_quotes(text):
    """Replace ConvoKit quote markers with standard quotation marks."""

    # Split text into lines for processing
    lines = text.split('\n')
    result_lines = []
    in_quote = False

    for line in lines:
        # Check if line starts a quote (begins with &gt;)
        if line.strip().startswith('&gt;'):
            if not in_quote:
                # Start of new quote - replace &gt; with opening quote
                cleaned_line = line.replace('&gt;', '"', 1).lstrip()
                result_lines.append(cleaned_line)
                in_quote = True
            else:
                # Continuation of quote - just remove &gt;
                cleaned_line = line.replace('&gt;', '', 1).lstrip()
                result_lines.append(cleaned_line)

        # Check if we're ending a quote (empty line or no more &gt; markers)
        elif in_quote and (line.strip() == '' or not line.strip().startswith('&gt;')):
            # End the quote by adding closing quote to previous line
            if result_lines and not result_lines[-1].strip().endswith('"'):
                result_lines[-1] = result_lines[-1].rstrip() + '"'

            # Add current line if it's not empty
            if line.strip():
                result_lines.append(line)
            else:
                result_lines.append(line)  # Keep empty lines

            in_quote = False

        else:
            # Regular line, not in quote
            result_lines.append(line)

    # Handle case where quote goes to end of text
    if in_quote and result_lines and not result_lines[-1].strip().endswith('"'):
        result_lines[-1] = result_lines[-1].rstrip() + '"'

    return '\n'.join(result_lines)

In [8]:
"""Stance detection functionality for conversation analysis."""
import numpy as np
import logging
from typing import List, Dict, Any, Optional
from tqdm import tqdm
from src.temporal_belief.utils.logger import setup_logging

logger = setup_logging("DEBUG")

class StanceDetector:
    """Detect political stance in ConvoKit utterances using BART."""

    def __init__(self, stance_labels: Optional[List[str]] = None,
                 config: ProjectConfig = None):
        """Initialize stance detector."""
        self.config = config or ProjectConfig()
        self.classifier = BARTZeroShotClassifier(self.config.bart_model_name)
        self.stance_labels = stance_labels or TOPIC_STANCE_HYPOTHESES
        logger.info(f"Initialized stance detector with labels: {self.stance_labels}")


    def detect_utterance_stance(self, utterance) -> Dict[str, Any]:
        """Detect stance for a single utterance."""
        # text = mark_quotes(utterance.text)
        if not utterance.text or utterance.text == '[removed]' or utterance.text == '[deleted]' or utterance.text.strip() == '.':
            logger.warning(f"No utterance found in {utterance.id}")
            return {"stance": "unknown", "confidence": 0.0}
        topic = utterance.meta['thread_topic']
        clean_text = prepare_text(utterance.text)
        text = get_contextual_framing_for_topic(topic, clean_text)
        # MAKE SURE THE TOPIC KEYS MATCH AND IF NOT RETURN A GENERAL
        stance_hypotheses = TOPIC_STANCE_HYPOTHESES.get(topic, GENERAL_STANCE_HYPOTHESES)
        logger.info(f"Text: {text}")
        #
        # # Pass one of the candidate stances
        # result = self.classifier.classify_text(text, self.stance_labels)
        # return {
        #     "stance": result["label"],
        #     "confidence": result["confidence"],
        #     "all_scores": result["all_scores"]
        # }

        confidence_threshold = 0.25
        stance_results = {}
        template_consistency_scores = {}

        for stance, hypotheses in stance_hypotheses.items():
            stance_scores = []

            # Test each hypothesis template for this stance
            for hypothesis in hypotheses:
                result = self.classifier.classify_text(text, [hypothesis])
                # Get score for this specific hypothesis
                stance_scores.append(result["all_scores"].get(hypothesis, 0.0))

            # Average across templates for this stance
            avg_confidence = np.mean(stance_scores)
            stance_results[stance] = avg_confidence

            # Measure consistency across templates (lower std = more consistent)
            template_consistency_scores[stance] = 1.0 - (np.std(stance_scores) / (np.mean(stance_scores) + 1e-8))

        # Find best stance
        best_stance = max(stance_results.keys(), key=lambda k: stance_results[k])
        best_confidence = stance_results[best_stance]
        overall_consistency = np.mean(list(template_consistency_scores.values()))

        # Apply confidence threshold
        if best_confidence < confidence_threshold:
            best_stance = 'neutral'
            best_confidence = stance_results.get('neutral', 0.0)

        return {
            'stance': best_stance,
            'confidence': best_confidence,
            'all_scores': stance_results,
            'method_used': 'multi_template_spinos',
            'template_consistency': overall_consistency,
            'reliable': best_confidence > confidence_threshold and overall_consistency > 0.7,
            'topic_context': topic
        }

    def process_corpus_utterances(self, corpus, batch_size: int = 50,
                              save_path: Optional[str] = None) -> None:
        """Process all utterances in corpus for stance detection."""
        utterances = list(corpus.iter_utterances())
        logger.info(f"Processing {len(utterances)} utterances for stance detection")

        for i in tqdm(range(0, len(utterances), batch_size),
                      desc="Processing utterances"):
            batch = utterances[i:i+batch_size]

            for utt in batch:
                try:
                    stance_result = self.detect_utterance_stance(utt)

                    # Add to utterance metadata
                    utt.add_meta("detected_stance", stance_result["stance"])
                    utt.add_meta("stance_confidence", stance_result["confidence"])
                    utt.add_meta("stance_scores", stance_result["all_scores"])

                except Exception as e:
                    logger.error(f"Failed to process utterance {utt.id}: {e}")
                    utt.add_meta("detected_stance", "unknown")
                    utt.add_meta("stance_confidence", 0.0)

        if save_path:
            corpus.dump(save_path)
            logger.info(f"Saved processed corpus to {save_path}")

        logger.info("Stance detection processing complete")

In [21]:
# Play around with a single part
# print(corpus.random_utterance().text)
stance_detector = StanceDetector()
utts = list(corpus.iter_utterances())
utt_text = utts[0].text
# utt_text = mark_quotes(utt_text)
# print(utt_text)
stance = stance_detector.detect_utterance_stance(utts[0])
print(stance['stance'])
# print(utts[1].meta[''])

Device set to use mps
2025-06-29 18:00:52,601 - temporal_belief.models.bart_classifier - INFO - bart_classifier:35 - Loaded BART model: facebook/bart-large-mnli on device: CPU
2025-06-29 18:00:52,607 - temporal_belief - INFO - 640296228:19 - Initialized stance detector with labels: {'abortion and reproductive rights': {'strongly_favor': ['The author strongly supports abortion rights and reproductive freedom', 'This comment advocates for unrestricted access to abortion services', 'The author expresses strong pro-choice activist viewpoints'], 'moderately_favor': ['The author moderately supports abortion rights with some limitations', 'This comment leans toward pro-choice but accepts some restrictions', 'The author supports reproductive rights with reasonable regulations'], 'neutral': ['This comment provides neutral information about abortion policy', 'The author presents balanced views on reproductive rights', 'This text discusses abortion without taking a clear stance'], 'moderately_aga

neutral


In [ ]:
# Testing 'process_corpus_utterances()'
SAVE_PATH = "/workspace/temporal_belief_analysis/pd_corpus_with_stances"
stance_detector = StanceDetector()
stance_detector.process_corpus_utterances(corpus, save_path=SAVE_PATH)

In [18]:
# Check if metadata gets added
utterances = list(corpus.iter_utterances())
first_utt = utterances[0]
print(f"First utterance ID: {first_utt.id}")
print(f"Has stance metadata: {'detected_stance' in first_utt.meta}")
if 'detected_stance' in first_utt.meta:
    print(f"Stance: {first_utt.meta['detected_stance']}")
    print(f"Confidence: {first_utt.meta['stance_confidence']}")

First utterance ID: nz333
Has stance metadata: False


In [ ]:
print()

In [9]:
# Testing 'detect_conversation_topic()' and 'dump()'
logger = setup_logging("DEBUG")
i = 0
utts = list(corpus.iter_utterances())
stance_detector = StanceDetector()
for i in range(30):
  logger.info("=" * 100)
  logger.info(f"Comment: {utts[i].text}\n")
  stance = stance_detector.detect_utterance_stance(utts[i])
  logger.info(f"Stance: {stance['stance']}")
  logger.info(f"Confidence: {stance['confidence']}")
  # logger.debug(f"Raw utterance text length: {len(utts[i].text)}")
  logger.warning(f"Low confidence score: {stance['confidence']}")
  # utts[i].add_meta("detected_stance", stance["stance"])
  # utts[i].add_meta("stance_confidence", stance["stance"])
  # utts[i].add_meta("stance_scores", stance["all_scores"])
  i += 1

del utts
# Test it saves properly:
# corpus_small.dump("/content/drive/MyDrive/MScProject/Corpora/corpus_small")


Device set to use mps
2025-06-29 18:09:06,547 - temporal_belief.models.bart_classifier - INFO - bart_classifier:35 - Loaded BART model: facebook/bart-large-mnli on device: CPU
2025-06-29 18:09:06,547 - temporal_belief - INFO - 3508063548:19 - Initialized stance detector with labels: {'abortion and reproductive rights': {'strongly_favor': ['The author strongly supports abortion rights and reproductive freedom', 'This comment advocates for unrestricted access to abortion services', 'The author expresses strong pro-choice activist viewpoints'], 'moderately_favor': ['The author moderately supports abortion rights with some limitations', 'This comment leans toward pro-choice but accepts some restrictions', 'The author supports reproductive rights with reasonable regulations'], 'neutral': ['This comment provides neutral information about abortion policy', 'The author presents balanced views on reproductive rights', 'This text discusses abortion without taking a clear stance'], 'moderately_ag